In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("C:/Users/Hp/Downloads/ADME_preprocessed_data.csv")

In [3]:
data.shape

(1739, 89)

In [4]:
(data.isnull().sum()/data.shape[0])*100

Unnamed: 0                0.0
LogS                      0.0
LogD                      0.0
LogP                      0.0
Pgp-inh                   0.0
                         ... 
PAINS                     0.0
Lipinski_encoded          0.0
Pfizer_encoded            0.0
GSK_encoded               0.0
GoldenTriangle_encoded    0.0
Length: 89, dtype: float64

In [5]:
(data.isnull().sum()/data.shape[0])*100

Unnamed: 0                0.0
LogS                      0.0
LogD                      0.0
LogP                      0.0
Pgp-inh                   0.0
                         ... 
PAINS                     0.0
Lipinski_encoded          0.0
Pfizer_encoded            0.0
GSK_encoded               0.0
GoldenTriangle_encoded    0.0
Length: 89, dtype: float64

In [6]:
data.duplicated().sum()

0

In [7]:
constant_features = [col for col in data.columns if len(data[col].unique()) == 1]
print(len(constant_features))

0


### feature selection

In [9]:
x=data.drop("LogP",axis=1)
y=data['LogP']

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=37)

In [11]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(x_train)  

VarianceThreshold(threshold=0)

In [12]:
non_const=sum(sel.get_support())  
non_const# 88 non constant features

88

In [17]:
sel = VarianceThreshold(threshold=0.1)  # 0.1 indicates 99% of observations approximately

sel.fit(X_train)

VarianceThreshold(threshold=0.1)

In [13]:

non_quassi=sum(sel.get_support()) 
non_quassi88 features are not quassi const

88

### finding the most imp feature-pearson

In [16]:
feature_target_corr = data.corr()['LogP']

# Exclude the target itself and get indices of features with correlation score above 0.80
features_above_threshold = feature_target_corr[abs(feature_target_corr) > 0.09].index

# Remove the target variable from the list
features_above_threshold = [feature for feature in features_above_threshold if feature != 'LogP']

print("Features with correlation score above 0.80 with the target:")
print(features_above_threshold)

Features with correlation score above 0.80 with the target:
['LogS', 'LogD', 'Pgp-inh', 'Pgp-sub', 'HIA', 'F(20%)', 'F(30%)', 'Caco-2', 'MDCK', 'BBB', 'PPB', 'VDss', 'Fu', 'CYP1A2-inh', 'CYP1A2-sub', 'CYP2C19-inh', 'CYP2C19-sub', 'CYP2C9-inh', 'CYP2C9-sub', 'CYP2D6-inh', 'CYP2D6-sub', 'CYP3A4-inh', 'CYP3A4-sub', 'CL', 'T12', 'hERG', 'DILI', 'ROA', 'FDAMDD', 'SkinSen', 'EC', 'Respiratory', 'BCF', 'IGC50', 'LC50', 'LC50DM', 'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'MW', 'Dense', 'nHA', 'nHD', 'TPSA', 'nRot', 'nRing', 'nHet', 'fChar', 'Flex', 'nStereo', 'NonBiodegradable', 'NonGenotoxic_Carcinogenicity', 'SureChEMBL', 'Acute_Aquatic_Toxicity', 'QED', 'Synth', 'Fsp3', 'Natural Product-likeness', 'BMS', 'Pfizer_encoded', 'GSK_encoded', 'GoldenTriangle_encoded']


### dropping high multicollinearity features through pearson feature selection

In [18]:

pearson_corr = data.corr(method='pearson')

upper_triangle = pearson_corr.where(np.triu(np.ones(pearson_corr.shape), k=1).astype(bool))
# Define a threshold for high correlation (e.g., 0.85)
threshold = 0.85

# Find columns to drop
columns_to_drop = set()
for col in upper_triangle.columns:
    # If any feature in a column is correlated above the threshold, mark it for removal
    if any(upper_triangle[col] > threshold):
        columns_to_drop.add(col)

print(len(columns_to_drop))
#Drop the highly correlated columns
final_data_reduced = data.drop(columns=columns_to_drop)

print(f"Features dropped: {columns_to_drop}")


7
Features dropped: {'nRot', 'nHD', 'LogP', 'nHet', 'TPSA', 'nHA', 'MCE-18'}


In [19]:
final_data_reduced.shape

(1739, 82)

In [20]:
final_data_reduced.to_excel("C:/Users/Hp/Documents/ project data/adme_reduced_data.xlsx")